## Set up connections

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json
import os

Connect to Kafka

In [ ]:
producer = KafkaProducer(bootstrap_servers=['replace with IP:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

Load data

In [ ]:
# for random only
df = pd.read_csv("path_to_some_.csv")

## Start streaming

### Testing random streaming to kafka and API

In [ ]:
# Random record
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)
    sleep(1)

In [ ]:
MY_SQL_API_URL='http://your-api-url/endpoint'

def random_publish_to_kafka(file_path,producer=producer,chunk_size=1000):
    """ 
    Send each row to MySQl API Kafka and
    """
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    chunk.append(next(reader))
                except StopIteration:
                    break
            if not chunk:
                break
            for row in chunk:
                # Convert row to JSON
                json_data = dumps(row)
                
                # Send row to MYSQL API
                try:
                    response = requests.post(MY_SQL_API_URL, json=json_data)
                    response.raise_for_status()
                    print(f"API response: {response.json()}")
                except Exception as e:
                    print(f"Error sending row to API: {e}")

                # Publish row to Kafka
                producer.send('invoice', value=bytes(json_data, 'utf-8'))
                sleep(1)
            producer.flush()
    producer.close()

In [ ]:
random_publish_to_kafka("path_to_some_.csv")

### Streaming in order from CSV

In [ ]:
SLEEP = 1

In [ ]:
def make_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (list, dict, str, int, float, bool, type(None))):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable value to string
    return serializable_dict

In [ ]:
def ordered_publish_to_kafka(file_path, producer=producer, chunk_size=1000):
    """ 
    Send each row to kafka mysql topic
    
    Uncomment either the environment code if using .env file or 
    variable code to set directly.
    
    SLEEP variable can be adjusted as desired. 1 is the minimum recommended setting. 
    Setting to 0 will crash the Kafka server unless a more robust EC2 setup is created.
    """
    try:
        with open(file_path, 'r') as file:
            reader = csv.DictReader(file)
            while True:
                chunk = []
                for _ in range(chunk_size):
                    try:
                        chunk.append(next(reader))
                    except StopIteration:
                        break
                if not chunk:
                    break
                for row in chunk:
                    # Convert row to JSON
                    row = make_serializable(row)
                    json_data = row

                    # Publish row to Kafka
                    producer.send('mysql', value=json_data)
                    producer.flush()
                    sleep(SLEEP)
    finally:
        producer.close()

In [ ]:
ordered_publish_to_kafka("path_to_some_.csv")

### Manually flush data when done

In [ ]:
producer.flush() #clear data from kafka server